# 간단한 RAG 소개

검색 증강 생성(Retrieval-Augmented Generation, RAG)은 정보 검색과 생성 모델을 결합한 하이브리드 접근 방식입니다. 외부 지식을 통합하여 언어 모델의 성능을 향상시켜 정확성과 사실적 정확성을 높입니다.

간단한 RAG 설정에서는 다음 단계를 따릅니다:

1. **데이터 수집 (Data Ingestion)**: 텍스트 데이터를 로드하고 전처리합니다. 이 단계에서는 다양한 소스에서 데이터를 가져와 RAG 시스템이 사용할 수 있는 형식으로 만듭니다.
2. **청킹 (Chunking)**: 검색 성능을 향상시키기 위해 데이터를 더 작은 청크(chunk)로 나눕니다. 너무 큰 텍스트 덩어리는 검색 효율성을 떨어뜨릴 수 있으므로, 의미 있는 단위로 잘게 쪼개는 과정이 중요합니다.
3. **임베딩 생성 (Embedding Creation)**: 임베딩 모델을 사용하여 텍스트 청크를 숫자 표현(벡터)으로 변환합니다. 이 숫자 표현은 텍스트의 의미를 담고 있어, 유사한 의미의 텍스트는 유사한 벡터 값을 갖게 됩니다.
4. **시맨틱 검색 (Semantic Search)**: 사용자 질의를 기반으로 관련 청크를 검색합니다. 사용자의 질문 또한 임베딩으로 변환되어, 저장된 텍스트 청크 벡터들과의 유사도를 비교하여 가장 관련성 높은 정보를 찾아냅니다.
5. **응답 생성 (Response Generation)**: 검색된 텍스트를 기반으로 언어 모델을 사용하여 응답을 생성합니다. 찾아낸 관련 정보를 바탕으로 사용자 질문에 대한 최종 답변을 만듭니다.

이 노트북은 간단한 RAG 접근 방식을 구현하고, 모델의 응답을 평가하며, 다양한 개선 사항을 살펴봅니다.

## 환경 설정
필요한 라이브러리를 가져오는 것으로 시작합니다.

In [1]:
import fitz  # PyMuPDF 라이브러리로 PDF 파일 처리에 사용됩니다.
import os
import numpy as np
import json
from openai import OpenAI

## PDF 파일에서 텍스트 추출
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 여기서는 PyMuPDF 라이브러리를 사용하여 PDF 파일에서 텍스트를 추출합니다.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출하고 처음 `num_chars`개의 문자를 출력합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화

    # PDF의 각 페이지 반복
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지 가져오기
        text = page.get_text("text")  # 페이지에서 텍스트 추출
        all_text += text  # 추출된 텍스트를 all_text 문자열에 추가

    return all_text  # 추출된 텍스트 반환

## 추출된 텍스트 청킹
추출된 텍스트가 있으면 검색 정확도를 높이기 위해 더 작고 중첩되는 청크로 나눕니다.
텍스트를 적절한 크기의 청크로 나누는 것은 RAG의 성능에 매우 중요합니다. 너무 작으면 문맥 정보가 부족하고, 너무 크면 관련 없는 정보가 많이 포함될 수 있습니다. 중첩(overlap)은 청크 간의 연속성을 유지하여 의미가 잘리는 것을 방지하는 데 도움이 됩니다.

In [3]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n개의 문자로 된 세그먼트로 나누고 중첩을 허용합니다.

    Args:
    text (str): 청킹할 텍스트.
    n (int): 각 청크의 문자 수.
    overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
    List[str]: 텍스트 청크 목록.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화
    
    # (n - overlap) 크기의 단계로 텍스트 반복
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트 청크를 청크 목록에 추가
        chunks.append(text[i:i + n])

    return chunks  # 텍스트 청크 목록 반환

## OpenAI API 클라이언트 설정
임베딩과 응답을 생성하기 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트 초기화
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 검색
)

## PDF 파일에서 텍스트 추출 및 청킹
이제 PDF를 로드하고, 텍스트를 추출한 다음, 청크로 분할합니다.

In [5]:
# PDF 파일 경로 정의
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트를 1000자 세그먼트로 청킹하고 200자 중첩 허용
text_chunks = chunk_text(extracted_text, 1000, 200)

# 생성된 텍스트 청크 수 출력
print("텍스트 청크 수:", len(text_chunks))

# 첫 번째 텍스트 청크 출력
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

Number of text chunks: 42

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

## 텍스트 청크에 대한 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.
각 텍스트 청크는 고차원 공간의 한 점으로 표현되며, 의미적으로 유사한 청크들은 이 공간에서 서로 가깝게 위치하게 됩니다. 이를 통해 관련 정보를 효과적으로 찾을 수 있습니다.

In [6]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    지정된 OpenAI 모델을 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩을 생성할 입력 텍스트.
    model (str): 임베딩 생성에 사용할 모델. 기본값은 "BAAI/bge-en-icl"입니다.

    Returns:
    dict: 임베딩을 포함하는 OpenAI API의 응답.
    """
    # 지정된 모델을 사용하여 입력 텍스트에 대한 임베딩 생성
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # 임베딩을 포함하는 응답 반환

# 텍스트 청크에 대한 임베딩 생성
response = create_embeddings(text_chunks)

## 시맨틱 검색 수행
사용자 질의에 가장 관련성 높은 텍스트 청크를 찾기 위해 코사인 유사도를 구현합니다.
코사인 유사도는 두 벡터 간의 각도의 코사인 값을 측정하여 벡터 간의 방향 유사성을 나타냅니다. 값이 1에 가까울수록 두 벡터(즉, 텍스트)는 의미적으로 유사하다고 판단합니다.

In [7]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 두 벡터 간의 코사인 유사도.
    """
    # 두 벡터의 내적을 계산하고 각 벡터의 노름(크기)의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [8]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    주어진 질의와 임베딩을 사용하여 텍스트 청크에 대한 시맨틱 검색을 수행합니다.

    Args:
    query (str): 시맨틱 검색을 위한 질의.
    text_chunks (List[str]): 검색할 텍스트 청크 목록.
    embeddings (List[dict]): 텍스트 청크에 대한 임베딩 목록.
    k (int): 반환할 상위 관련 텍스트 청크 수. 기본값은 5입니다.

    Returns:
    List[str]: 질의를 기반으로 한 상위 k개의 가장 관련성 높은 텍스트 청크 목록.
    """
    # 질의에 대한 임베딩 생성
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # 유사도 점수를 저장할 리스트 초기화

    # 질의 임베딩과 각 텍스트 청크 임베딩 간의 유사도 점수 계산
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, similarity_score))  # 인덱스와 유사도 점수 추가

    # 유사도 점수를 내림차순으로 정렬
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # 상위 k개의 가장 유사한 텍스트 청크의 인덱스 가져오기
    top_indices = [index for index, _ in similarity_scores[:k]]
    # 상위 k개의 가장 관련성 높은 텍스트 청크 반환
    return [text_chunks[index] for index in top_indices]


## 추출된 청크에 대한 질의 실행

In [9]:
# JSON 파일에서 검증 데이터 로드
with open('data/val.json') as f:
    data = json.load(f)

# 검증 데이터에서 첫 번째 질의 추출
query = data[0]['question']

# 시맨틱 검색을 수행하여 질의에 대해 상위 2개의 가장 관련성 높은 텍스트 청크 찾기
top_chunks = semantic_search(query, text_chunks, response.data, k=2)

# 질의 출력
print("질의:", query)

# 상위 2개의 가장 관련성 높은 텍스트 청크 출력
for i, chunk in enumerate(top_chunks):
    print(f"문맥 {i + 1}:\n{chunk}\n=====================================")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:
systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy. 
Privacy and Data Protection 
AI systems often rely on large amounts of data, raising concerns about privacy and data 
protection. Ensuring responsible data handling, implementing privacy-preserving techniques, 
and complying with data protection regulations are crucial. 
Accountability and Responsibility 
Establishing accountability and responsibility for AI systems is essential for addressing potential 
harms and ensuring ethical behavior. This includes defining roles and responsibilities for 
developers, deployers, and users of AI systems. 
Chapter 20: Building Trust in AI 
Transparency and Explainability 
Transparency and explainability are key to building trust in AI. Making AI systems understandable 
and providing insights into their decision-making processes he

## 검색된 청크를 기반으로 응답 생성

In [10]:
# AI 어시스턴트에 대한 시스템 프롬프트 정의
system_prompt = "당신은 주어진 문맥에 기반하여 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하십시오."

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델로부터 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동을 안내하는 시스템 프롬프트.
    user_message (str): 사용자의 메시지 또는 질의.
    model (str): 응답 생성에 사용할 모델. 기본값은 "meta-llama/Llama-3.2-3B-Instruct"입니다. (원문: "meta-llama/Llama-2-7B-chat-hf")

    Returns:
    dict: AI 모델의 응답.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0, # 낮은 온도는 더 결정적이고 일관된 답변을 생성합니다.
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 상위 청크를 기반으로 사용자 프롬프트 생성
user_prompt = "\n".join([f"문맥 {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\n질문: {query}"

# AI 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

## AI 응답 평가
AI 응답을 예상 답변과 비교하고 점수를 매깁니다.
RAG 시스템의 성능을 평가하는 것은 매우 중요합니다. 여기서는 간단하게 미리 정의된 '이상적인 답변'과 AI가 생성한 답변을 비교하여 점수를 매기지만, 실제로는 더 정교한 평가 지표(예: ROUGE, BLEU, 의미적 유사도 기반 점수 등)를 사용하거나, 인간 평가자가 직접 평가하기도 합니다.

In [11]:
# 평가 시스템에 대한 시스템 프롬프트 정의
evaluate_system_prompt = "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. AI 어시스턴트의 응답이 실제 응답과 매우 유사하면 1점을 부여하십시오. 응답이 실제 응답과 비교하여 부정확하거나 만족스럽지 않으면 0점을 부여하십시오. 응답이 실제 응답과 부분적으로 일치하면 0.5점을 부여하십시오."

# 사용자 질의, AI 응답, 실제 응답 및 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
evaluation_prompt = f"사용자 질의: {query}\nAI 응답:\n{ai_response.choices[0].message.content}\n실제 응답: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답 생성
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 응답 출력
print(evaluation_response.choices[0].message.content)

Based on the evaluation criteria, I would assign a score of 0.8 to the AI assistant's response.

The AI assistant's response is very close to the true response, but there are some minor differences. The true response mentions "transparency" and "accountability" explicitly, which are not mentioned in the AI assistant's response. However, the overall meaning and content of the response are identical, and the AI assistant's response effectively conveys the importance of Explainable AI in building trust and ensuring fairness in AI systems.

Therefore, the score of 0.8 reflects the AI assistant's response being very close to the true response, but not perfectly aligned.
